In [1]:
!pip install azure-storage-blob
!pip install pyarrow
!pip install psycopg2 sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine
import psycopg2
from datetime import datetime

In [3]:
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

In [24]:
#specify the path to your JSON configuration file
config_file_path = 'config.json'

#load with JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE = 'bic'

#initialize the blobserviceclient
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

#get the container client
container_client = blob_service_client.get_container_client(CONTAINER_AZURE)

bic_df = pd.DataFrame()

#list all blobs in specified container
blob_list = container_client.list_blobs()
for blob in blob_list:
  print(blob.name)
  blob_client = container_client.get_blob_client(blob=blob.name)
  blob_data = blob_client.download_blob()
  blob_content = blob_data.readall().decode('utf-8')
  df = pd.read_csv(StringIO(blob_content))
  #display the head of the dataframe
  print(df.shape)
  bic_df = df.copy()



bic.csv
(10000, 31)
bic_20240503.csv
(10000, 31)


In [25]:
bic_df.head()

,violation_number,bic_number,account_name,type_of_violation,violation_phase,violation_disposition,account_type,violation_account_address,violation_account_city,violation_account_state,...,description_of_rule,export_date,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta
0,TWC-210067,269.0,JOE'S WASTE PAPER CORP.,OATH,Issued,Settled,License,P.O. BOX 320358,BROOKLYN,NY,...,Failed to timely disclose to Commission employ...,2024-05-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TWC-223791,NaN,ROCK E SMALL PLUMBING & HEATING INC,OATH,Issued,Settled,Unlicensed,152-32 Rockaway Blvd,Jamaica,NY,...,Removed collected or disposed of trade waste o...,2024-05-03,40.671873,-73.784991,12.0,28.0,294.0,4263107.0,4.121250e+09,Springfield Gardens North
2,TWC-222099,NaN,SCLAFMORE CONSTRUCTION LLC,OATH,Issued,Settled,Unlicensed,69-37 50th Avenue,Queens,NY,...,Removed collected or disposed of trade waste o...,2024-05-03,40.736912,-73.894918,2.0,30.0,489.0,4056355.0,4.024350e+09,Elmhurst-Maspeth
3,TWC-226916,1240.0,ROYAL WASTE SERVICES INC.,OATH,Issued,Settled,License,168-46 DOUGLAS AVENUE,JAMAICA,NY,...,A trade waste vehicle must not be operated unl...,2024-05-03,40.703949,-73.789576,12.0,27.0,444.0,NaN,NaN,Jamaica
4,TWC-227078,NaN,Izzy Landscaper Inc,OATH,Issued,Settled,Unlicensed,110 Brightside Avenue,Central Islip,NY,...,"It shall be unlawful for any person to remove,...",2024-05-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
def week_of_month(dt):
    first_day = dt.replace(day=1)
    dom = dt.day
    adjusted_dom = dom + first_day.weekday()
    return int(adjusted_dom/7.0)

def get_week_of_year(date_str):
    """
    Calculate the ISO week number of the year for a given date.

    Parameters:
    date_str (str): A date string in the format 'YYYY-MM-DD'.

    Returns:
    int: The ISO week number of the year.
    """
    #parse the input string to a datetime object
    date = datetime.strptime(date_str, '%Y-%m-%d')

    #get the ISO calendar week number
    week_of_year = date.isocalendar()[1]

    return week_of_year


In [27]:
#cleaning
bic_df['bic_number'] = bic_df['bic_number'].astype('Int64')
bic_df['community_board'] = bic_df['community_board'].astype('Int64')
bic_df['census_tract'] = bic_df['census_tract'].astype('Int64')
bic_df['bin'] = bic_df['bin'].astype('Int64')
bic_df['bbl'] = bic_df['bbl'].astype('Int64')

bic_df.head()


,violation_number,bic_number,account_name,type_of_violation,violation_phase,violation_disposition,account_type,violation_account_address,violation_account_city,violation_account_state,...,description_of_rule,export_date,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta
0,TWC-210067,269,JOE'S WASTE PAPER CORP.,OATH,Issued,Settled,License,P.O. BOX 320358,BROOKLYN,NY,...,Failed to timely disclose to Commission employ...,2024-05-03,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,NaN
1,TWC-223791,<NA>,ROCK E SMALL PLUMBING & HEATING INC,OATH,Issued,Settled,Unlicensed,152-32 Rockaway Blvd,Jamaica,NY,...,Removed collected or disposed of trade waste o...,2024-05-03,40.671873,-73.784991,12,28.0,294,4263107,4121250032,Springfield Gardens North
2,TWC-222099,<NA>,SCLAFMORE CONSTRUCTION LLC,OATH,Issued,Settled,Unlicensed,69-37 50th Avenue,Queens,NY,...,Removed collected or disposed of trade waste o...,2024-05-03,40.736912,-73.894918,2,30.0,489,4056355,4024350036,Elmhurst-Maspeth
3,TWC-226916,1240,ROYAL WASTE SERVICES INC.,OATH,Issued,Settled,License,168-46 DOUGLAS AVENUE,JAMAICA,NY,...,A trade waste vehicle must not be operated unl...,2024-05-03,40.703949,-73.789576,12,27.0,444,<NA>,<NA>,Jamaica
4,TWC-227078,<NA>,Izzy Landscaper Inc,OATH,Issued,Settled,Unlicensed,110 Brightside Avenue,Central Islip,NY,...,"It shall be unlawful for any person to remove,...",2024-05-03,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,NaN


In [28]:
dim_account = pd.DataFrame({
    'account_id': range(1, len(bic_df) + 1),  # Assigning a unique ID for each entry
    'bic_number': bic_df['bic_number'],
    'account_name': bic_df['account_name'],
    'account_type': bic_df['account_type']
})

# Preview the created dimension table
dim_account.head()

,account_id,bic_number,account_name,account_type
0,1,269,JOE'S WASTE PAPER CORP.,License
1,2,<NA>,ROCK E SMALL PLUMBING & HEATING INC,Unlicensed
2,3,<NA>,SCLAFMORE CONSTRUCTION LLC,Unlicensed
3,4,1240,ROYAL WASTE SERVICES INC.,License
4,5,<NA>,Izzy Landscaper Inc,Unlicensed


In [29]:
dim_violation = pd.DataFrame({
    'violation_id': range(1, len(bic_df) + 1),
    'violation_number': bic_df['violation_number'],
    'type_violation': bic_df['type_of_violation'],
    'violation_phase': bic_df['violation_phase']
})

# Preview the created dimension table
dim_violation.head()

,violation_id,violation_number,type_violation,violation_phase
0,1,TWC-210067,OATH,Issued
1,2,TWC-223791,OATH,Issued
2,3,TWC-222099,OATH,Issued
3,4,TWC-226916,OATH,Issued
4,5,TWC-227078,OATH,Issued


In [30]:
dim_location = pd.DataFrame({
    'location_id': range(1, len(bic_df) + 1),
    'borough': bic_df['borough_of_violation'],  # Assuming this column exists in your DataFrame
    'latitude': bic_df['latitude'],
    'longitude': bic_df['longitude']
})

# Preview the created dimension table
dim_location.head()

,location_id,borough,latitude,longitude
0,1,Manhattan,NaN,NaN
1,2,Brooklyn,40.671873,-73.784991
2,3,Brooklyn,40.736912,-73.894918
3,4,Bronx,40.703949,-73.789576
4,5,Queens,NaN,NaN


In [31]:
bic_df['date_of_violation'] = pd.to_datetime(bic_df['date_of_violation'])


dim_date = pd.DataFrame({
    'date_id': np.arange(1, len(bic_df) + 1),
    'Date_original': bic_df['date_of_violation'],
    'date_Iso': bic_df['date_of_violation'].dt.date,
    'year': bic_df['date_of_violation'].dt.year,
    'quarter': bic_df['date_of_violation'].dt.quarter,
    'month': bic_df['date_of_violation'].dt.month,
    'month_name': bic_df['date_of_violation'].dt.strftime('%B'),
    'week_of_month': bic_df['date_of_violation'].apply(lambda x: (x.day - 1) // 7 + 1),
    'week_of_year': bic_df['date_of_violation'].dt.isocalendar().week,
    'day_number': bic_df['date_of_violation'].dt.day,
    'day_of_week': bic_df['date_of_violation'].dt.weekday + 1,
    'day_name': bic_df['date_of_violation'].dt.strftime('%A')
})



print(dim_date.head())

   date_id       Date_original    date_Iso  year  quarter  month month_name  \
0        1 2014-06-25 04:00:00  2014-06-25  2014        2      6       June   
1        2 2022-04-19 04:00:00  2022-04-19  2022        2      4      April   
2        3 2021-07-06 04:00:00  2021-07-06  2021        3      7       July   
3        4 2023-06-16 04:43:00  2023-06-16  2023        2      6       June   
4        5 2023-07-05 14:34:00  2023-07-05  2023        3      7       July   

   week_of_month  week_of_year  day_number  day_of_week   day_name  
0              4            26          25            3  Wednesday  
1              3            16          19            2    Tuesday  
2              1            27           6            2    Tuesday  
3              3            24          16            5     Friday  
4              1            27           5            3  Wednesday  


In [32]:
facts_fine = pd.DataFrame({
    'fact_id': range(1, len(bic_df) + 1),
    'fine_amount': bic_df['fine_amount'],
    'maximum_fine': bic_df['maximum_fine'],
    'violation_number': bic_df['violation_number'],  # Assuming direct mapping from bic_df
    'account_id': dim_account['account_id'],  # Linking account_id from dim_account
    'violation_id': dim_violation['violation_id'],  # Linking violation_id from dim_violation
    'location_id': dim_location['location_id'],  # Linking location_id from dim_location
    'date_id': dim_date['date_id']  # Linking date_id from dim_date
})

# Preview the created facts table
print(facts_fine.head())

   fact_id  fine_amount  maximum_fine violation_number  account_id  \
0        1        250.0       10000.0       TWC-210067           1   
1        2          0.0        5000.0       TWC-223791           2   
2        3       1000.0        5000.0       TWC-222099           3   
3        4          0.0           0.0       TWC-226916           4   
4        5        500.0           0.0       TWC-227078           5   

   violation_id  location_id  date_id  
0             1            1        1  
1             2            2        2  
2             3            3        3  
3             4            4        4  
4             5            5        5  
